<a href="https://colab.research.google.com/github/heghiw/DeepLearningPetIdentification/blob/main/baseline_exper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 1st -  Download utility.py file from github repository
## 2nd - Imports all functions from utility.py

import requests

# Correct raw URL for the utility.py file
url = "https://raw.githubusercontent.com/avkaz/DeepLearningPetIdentification/main/utility.py"

# Fetch and save the file locally
response = requests.get(url)
with open("utility.py", "wb") as f:
    f.write(response.content)


import utility
print("utility.py downloaded successfully.")


utility.py downloaded successfully.


In [2]:
data = utility.get_data()

In [18]:
trypets = list(data.items())[:20]

In [19]:
trypets

[('tanyny-chomutov-2024-12-21',
  {'Jméno': 'Tanyny',
   'Pohlaví': 'Samec',
   'Kraj': 'Ústecký',
   'Okres': 'Chomutov',
   'Plemeno': 'Kříženec',
   'Věk': '5 let',
   'Barva': 'Černá',
   'Velikost': 'Střední - 10-17kg',
   'url': 'https://www.psidetektiv.cz/zvire/tanyny-chomutov-2024-12-21',
   'images': ['https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190390.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190391.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190392.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190393.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190394.jpg']}),
 ('haily-tachov-2024-12-21',
  {'Jméno': 'Haily',
   'Pohlaví': 'Samice',
   'Kraj': 'Plzeňský',
   'Okres': 'Tachov',
   'Plemeno': 'Jezevčík',
   'Věk': '5 let',
   'Barva': 'Tmavý divočák',
   'Velikost': 'Střední - 10-17kg',
   'url': 'https://www.psidetektiv.cz/zvire/haily-tachov-2024-12-21',
   'images

In [36]:
import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

In [65]:
import requests
import json
import numpy as np
import tensorflow as tf
from PIL import Image
from io import BytesIO
import pandas as pd
from google.colab import files
!pip install faiss-gpu
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import faiss


cleaning data/ deleting instances with multiple pets in one posting

In [20]:
f_trypets = [
    pet for pet in trypets
    if not any(part.startswith('a') for part in pet[0].split('-')[1:])
]

In [28]:

print(len(f_trypets))

17


In [29]:
f_trypets

[('tanyny-chomutov-2024-12-21',
  {'Jméno': 'Tanyny',
   'Pohlaví': 'Samec',
   'Kraj': 'Ústecký',
   'Okres': 'Chomutov',
   'Plemeno': 'Kříženec',
   'Věk': '5 let',
   'Barva': 'Černá',
   'Velikost': 'Střední - 10-17kg',
   'url': 'https://www.psidetektiv.cz/zvire/tanyny-chomutov-2024-12-21',
   'images': ['https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190390.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190391.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190392.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190393.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190394.jpg']}),
 ('haily-tachov-2024-12-21',
  {'Jméno': 'Haily',
   'Pohlaví': 'Samice',
   'Kraj': 'Plzeňský',
   'Okres': 'Tachov',
   'Plemeno': 'Jezevčík',
   'Věk': '5 let',
   'Barva': 'Tmavý divočák',
   'Velikost': 'Střední - 10-17kg',
   'url': 'https://www.psidetektiv.cz/zvire/haily-tachov-2024-12-21',
   'images

create embeddings

In [86]:
# embeddings for FAISS
all_embeddings = []
all_pet_names = []
all_image_urls = []

for pet_id, pet_info in f_trypets:
    pet_name = pet_info['Jméno']
    for image_url in pet_info["images"]:
        embedding = utility.download_and_preprocess_image(image_url, target_size=(224, 224))
        if embedding is not None:
            all_embeddings.append(embedding.numpy().flatten())  # Flatten the embedding and add to the list
            all_pet_names.append(pet_name)
            all_image_urls.append(image_url)

all_embeddings = np.array(all_embeddings)

faiss

In [96]:
def normalize_embeddings(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms


# FAISS
l2_index = faiss.IndexFlatL2(all_embeddings.shape[1])  # L2

cosine_similarity_index = faiss.IndexFlatIP(all_embeddings.shape[1])  # cos

# normalized for cos similarity
normalized_embeddings = normalize_embeddings(all_embeddings)

# add to the indexes
l2_index.add(all_embeddings)

cosine_similarity_index.add(normalized_embeddings)

# test for tanyny
test_image_embedding = all_embeddings[0:1]
test_image_embedding_normalized = normalized_embeddings[0:1]

# FAISS search
l2_distances, l2_indices = l2_index.search(test_image_embedding, k=5)
cosine_sim_scores, cosine_sim_indices = cosine_similarity_index.search(test_image_embedding_normalized, k=5)

results = []

# l2 res
for idx, dist in zip(l2_indices[0], l2_distances[0]):
    results.append(("Tanyny", all_pet_names[idx], "L2 Distance", dist))


# cos sim res
for idx, score in zip(cosine_sim_indices[0], cosine_sim_scores[0]):
    results.append(("Tanyny", all_pet_names[idx], "Cosine Similarity", score))

#df
df = pd.DataFrame(
    results,
    columns=["Test Pet", "Matched Pet", "Metric", "Similarity Score"]
)

print("\nTop 5 Matches for Tanyny's First Image Using Different Metrics:\n", df)

# save as excel
#df.to_excel("top5_matches_tanyny.xlsx", index=False)
#files.download("top5_matches_tanyny.xlsx")



Top 5 Matches for Tanyny's First Image Using Different Metrics:
   Test Pet Matched Pet             Metric  Similarity Score
0   Tanyny      Tanyny        L2 Distance          0.000000
1   Tanyny      Tanyny        L2 Distance       6617.147949
2   Tanyny       Haily        L2 Distance       7191.615723
3   Tanyny      Tanyny        L2 Distance      12555.998047
4   Tanyny      Tanyny        L2 Distance      19516.195312
5   Tanyny      Tanyny  Cosine Similarity          0.999968
6   Tanyny      Tanyny  Cosine Similarity          0.841105
7   Tanyny       Haily  Cosine Similarity          0.804892
8   Tanyny      Tanyny  Cosine Similarity          0.774255
9   Tanyny      Tanyny  Cosine Similarity          0.742521
